In [2]:
import pandas as pd
import numpy as np

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np

X = train.drop(columns=['number_of_rentals', 'date_time'])
y = train['number_of_rentals']

# train, valid 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# X 데이터만 스케일링하기 (y는 스케일링하지 않음)
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_valid_scaled = sc.transform(X_valid)
X_test_scaled = sc.transform(test.drop(columns=['date_time']))

# 모델 학습 (훈련 데이터로만!)
lasso_model = Lasso(alpha=1.0, random_state=42)
lasso_model.fit(X_train_scaled, y_train)

# 예측 (검증 데이터로만 평가)
y_train_pred = lasso_model.predict(X_train_scaled)
y_valid_pred = lasso_model.predict(X_valid_scaled)

# NMAE 계산 (y를 스케일링하지 않았으니 원래 값으로 비교하면 됨)
nmae_train = mean_absolute_error(y_train, y_train_pred) / np.mean(y_train)
nmae_valid = mean_absolute_error(y_valid, y_valid_pred) / np.mean(y_valid)

print('train nmae:', nmae_train)
print('valid nmae:', nmae_valid)


train nmae: 0.3051162501619611
valid nmae: 0.3021020230072927


c:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.281e+07, tolerance: 1.566e+07
  model = cd_fast.enet_coordinate_descent(


In [9]:
lasso_model.fit(X_train_scaled, y_train)
important_features = np.array(X.columns)[lasso_model.coef_ != 0]
print("선택된 특성들:", important_features)

선택된 특성들: ['wind_direction' 'sky_condition' 'precipitation_form' 'wind_speed'
 'humidity' 'low_temp' 'high_temp' 'Precipitation_Probability']


c:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.281e+07, tolerance: 1.566e+07
  model = cd_fast.enet_coordinate_descent(


In [10]:
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lasso_model.coef_
})

# 절댓값 기준 내림차순 정렬
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))

                     Feature   Coefficient
7  Precipitation_Probability -27122.783240
2         precipitation_form  11219.499567
1              sky_condition  10897.874633
6                  high_temp   8799.269525
3                 wind_speed  -7820.645055
0             wind_direction   4020.648589
5                   low_temp  -2578.508385
4                   humidity   1915.646916
